<a href="https://colab.research.google.com/github/MalVaroTor/MalVaroTor/blob/main/TCC_3_SCRIPT_CALCULOS_MASSA_ENERGIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SCRIPT DE CÁLCULO DA MASSA**

---



In [ ]:
#   IMPORTANDO O ARQUIVO E LENDO A TABELA ESPECIFICADA.


import pandas as pd                 # IMPORTANDO BIBLIOTECA PARA DADOS

import matplotlib.pyplot as plt     # IMPORTANDO BIBLIOTECA PARA GRÁFICOS.

import numpy as np                  # IMPOTANDO BIBLIOTECA PARA TRABALHAR COM NÚMEROS.

import matplotlib.dates as mdates

planilha = input("Digite o nome da planilha: ")

db = pd.read_excel('/content/drive/MyDrive/TCC/db_dados/dados_2023dez.xlsx', \
                        sheet_name = planilha)

display(db)

# ------------------------------------------------------------------------------------------------------------------------
#   FILTRANDO A TABELA PARA A DATA 2023-12-07, NOS HORÁRIOS ESPECÍFICOS DE POWERSET DE ESTUDO DA TABELA dados_I_V_07dez.

# Filtrar a data

data_estudo = input("Digite a data de estudo no formato 'AAAA-MM-DD': ")

display(db)

# Filtrar os dados para a data específica
dados_data_especifica = db[db['Data/Hora'].dt.date == pd.to_datetime(data_estudo).date()]

# Especificar o intervalo de horário
hora_inicio = input("Digite a hora inicial do PowerSet de produção no formato 'HH:MM:SS': ")

hora_fim = input("Digite a hora final do PowerSet de produção no formato 'HH:MM:SS': ")

# Filtrar os dados dentro do intervalo de horário
dados_07dez = dados_data_especifica[(dados_data_especifica['Data/Hora'].dt.time >= pd.to_datetime(hora_inicio).time()) &
                                        (dados_data_especifica['Data/Hora'].dt.time <= pd.to_datetime(hora_fim).time())]

display(dados_07dez)

# ------------------------------------------------------------------------------------------------------------------------
#   CRIANDO A COLUNA DE HORAS EM FORMATO DECIMAL.

dados_07dez['Hora_decimal'] = dados_07dez['Data/Hora'].dt.hour + \
                                dados_07dez['Data/Hora'].dt.minute / 60 + \
                                    dados_07dez['Data/Hora'].dt.second / 3600

display(dados_07dez)

# ------------------------------------------------------------------------------------------------------------------------
#   DELETANDO AS LINHAS A QUAL A COLUNA M_Fluxo_[Nm3/h] SE ENCONTRA VAZIA.

dados_07dez = dados_07dez.dropna(subset=['M_Fluxo_[Nm3/h]'])

display(dados_07dez)

# ------------------------------------------------------------------------------------------------------------------------
#   ACRÉSCIMOS DE VALORES DE FLUXO DE MASSA DEVIDO A VALORES BASES ABAIXO DE ZERO.

# Adicionar o valor 0.42422 aos dados da coluna 'M_Fluxo_[Nm3/h]'
incremento = input("Digite o incremento de dados do Fluxo de Massa em número decimal: ")

incremento = float(incremento)

dados_07dez['M_Fluxo_[Nm3/h]'] += incremento

display(dados_07dez)

# ------------------------------------------------------------------------------------------------------------------------
#   # **CALCULANDO A INTEGRAL DE Nm3/h EM RELAÇÃO A h, REFERENTE AO GRÁFICO.

# Calcular a integral numérica dos dados de fluxo em relação às horas decimais usando o método do trapézio
integral_numerica = np.trapz(dados_07dez['M_Fluxo_[Nm3/h]'], \
                                x=dados_07dez['Hora_decimal'])

qdd_H2 = integral_numerica * 0.0899

kgH2_anterior = input("Digite de H2 anterior em número decimal: : ")

kgH2_anterior = float(kgH2_anterior)

comparacao =  kgH2_anterior / qdd_H2

# ------------------------------------------------------------------------------------------------------------------------
#    APRESENTAÇÃO DOS DADOS.

print("Quantidade de Fluxo de Massa em Nm3/h: ", integral_numerica)

print("Quantidade de H2 produzido em kg: ", qdd_H2)

print("Relação de antes e depois H2/H2: ", comparacao)

comparacao = comparacao * 100

comparacao_format = "{:.3f}%".format(comparacao)

print("Relação Percentual de antes e depois H2/H2: ",comparacao_format)

# **SCRIPT DO CÁLCULO DA ENERGIA**

---



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
import statistics
db = pd.read_excel('/content/drive/MyDrive/TCC/db_dados/all_dados_I_V_T.xlsx')
db['Data'] = pd.to_datetime(db['Data'])
db['Data'] = db['Data'].dt.strftime('%d/%m/%Y')
# ------------------------------------------------------------------------------------------------------------------------
db_data_estudo = input('Digite a data de operação: ')
var_data_estudo = db_data_estudo
var_data_estudo = pd.to_datetime(var_data_estudo)
var_data_estudo = var_data_estudo.strftime('%d/%m/%Y')

db_data_estudo = pd.to_datetime(db_data_estudo)
db_data_estudo = db[db['Data/Hora'].dt.date == pd.to_datetime(db_data_estudo).date()]

display(db_data_estudo)
# ------------------------------------------------------------------------------------------------------------------------
ps_entrada = input('Digite o PowerSet de produção de H2: ')
ps_entrada = float(ps_entrada)
db_ps_estudo = db_data_estudo.loc[db_data_estudo['PowerSet_[%]'] >= ps_entrada]

display(db_ps_estudo)
# ------------------------------------------------------------------------------------------------------------------------
indice_tensao_A = db_ps_estudo.columns.get_loc('Tensao_A_[V]')
db_ps_estudo.insert(indice_tensao_A + 1, \
                        'Potencia_A_[kW]', \
                            db_ps_estudo['Corrente_A_[A]'] * db_ps_estudo['Tensao_A_[V]'] / 1000)
indice_tensao_B = db_ps_estudo.columns.get_loc('Tensao_B_[V]')
db_ps_estudo.insert(indice_tensao_B + 1, \
                        'Potencia_B_[kW]', \
                            db_ps_estudo['Corrente_B_[A]'] * db_ps_estudo['Tensao_B_[V]'] / 1000)
# ------------------------------------------------------------------------------------------------------------------------
db_ps_estudo['Hora_decimal'] = db_ps_estudo['Data/Hora'].dt.hour +                              \
                                    db_ps_estudo['Data/Hora'].dt.minute /                       \
                                            60 + db_ps_estudo['Data/Hora'].dt.second / 3600
# ------------------------------------------------------------------------------------------------------------------------

db_ps_estudo = db_ps_estudo.dropna(subset=['Potencia_A_[kW]', 'Potencia_B_[kW]'])

energia_a = np.trapz(db_ps_estudo['Potencia_A_[kW]'], \
                        x=db_ps_estudo['Hora_decimal'])
energia_a = round(energia_a, 3)
energia_b = np.trapz(db_ps_estudo['Potencia_B_[kW]'], \
                        x=db_ps_estudo['Hora_decimal'])

energia_b = round(energia_b, 3)
print(f"Quantidade de Energia do Eletrolisador A em [kWh] no dia {var_data_estudo}: ", energia_a)
print(f"Quantidade de Energia do Eletrolisador B em [kWh] no dia {var_data_estudo}: ", energia_b)
energia_total_1 = energia_a + energia_b
print(f"Quantidade Total de Energia em [kWh] no dia {var_data_estudo}: ", energia_total_1)
# ------------------------------------------------------------------------------------------------------------------------
print("------------------------------------------------------------------------------------------------------")
# ------------------------------------------------------------------------------------------------------------------------
hora_inicio = input("Digite a hora inicial do PowerSet de produção no formato 'HH:MM:SS': ")
hora_fim = input("Digite a hora final do PowerSet de produção no formato 'HH:MM:SS': ")
# Filtrar os dados dentro do intervalo de horário
db_hr_estudo = db_ps_estudo[(db_ps_estudo['Data/Hora'].dt.time >= pd.to_datetime(hora_inicio).time()) &
                                                (db_ps_estudo['Data/Hora'].dt.time <= pd.to_datetime(hora_fim).time())]

display(db_hr_estudo)

# ------------------------------------------------------------------------------------------------------------------------
energia_a_hr = np.trapz(db_hr_estudo['Potencia_A_[kW]'], x=db_hr_estudo['Hora_decimal'])
energia_a_hr = round(energia_a_hr, 3)
energia_b_hr = np.trapz(db_hr_estudo['Potencia_B_[kW]'], x=db_hr_estudo['Hora_decimal'])
energia_b_hr = round(energia_b_hr, 3)
print(f"Quantidade de Energia do Eletrolisador A em [kWh] no dia {var_data_estudo}: ", energia_a_hr)
print(f"Quantidade de Energia do Eletrolisador B em [kWh] no dia {var_data_estudo}: ", energia_b_hr)
energia_total = energia_a_hr + energia_b_hr
print(f"Quantidade Total de Energia em [kWh] no dia {var_data_estudo}: ", energia_total)